In [5]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K


In [6]:
import numpy as np
import pandas as pd
import random
import os
import re
import string
import warnings
import cv2
import email
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from imutils import paths
from bs4 import BeautifulSoup
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (accuracy_score, f1_score, confusion_matrix, 
                             precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from wordcloud import WordCloud

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Activation, 
                                     Flatten, Dense)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

warnings.filterwarnings('ignore')
np.random.seed(49)

In [7]:
import pickle

with open('email_texts.pkl', 'rb') as f:
    email_texts = pickle.load(f)

with open('label_list.pkl', 'rb') as f:
    label_list = pickle.load(f)

print("Data loaded from pickle files.")

X_train, X_test, y_train, y_test = train_test_split(email_texts, label_list, test_size=0.1, random_state=42)

Data loaded from pickle files.


In [8]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(X_train)

CountVectorizer(stop_words='english')

In [9]:
X_train = vectorizer.transform(X_train).toarray()
y_train = np.array(y_train).reshape(len(y_train), 1)

In [10]:
X_test = vectorizer.transform(X_test).toarray()
y_test = np.array(y_test).reshape(len(y_test), 1)

In [11]:
class SimpleMLP:
    @staticmethod
    def build(shape):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(1))
        model.add(Activation("sigmoid"))

        return model

In [12]:
def add_laplace_noise(data, sensitivity, epsilon):
    scale = sensitivity / epsilon
    noise = np.random.laplace(0, scale, data.shape)
    noisy_data = data + noise
    return noisy_data

In [ ]:
def evaluate_utility(noisy_X_train, noisy_X_test, y_train, y_test):
    model = SimpleMLP.build(noisy_X_train.shape[1])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(noisy_X_train, y_train, epochs=10, verbose=0)
    y_pred = (model.predict(noisy_X_test) > 0.5).astype(int)

    # Calculate accuracy
    return accuracy_score(y_test, y_pred)

In [ ]:
sensitivity = 1
epsilon_values = np.linspace(0.1, 10, 30)
accuracies = []


for epsilon in epsilon_values:
    noisy_X_train = add_laplace_noise(X_train, sensitivity, epsilon)
    noisy_X_test = add_laplace_noise(X_test, sensitivity, epsilon)
    accuracy = evaluate_utility(noisy_X_train, noisy_X_test, y_train, y_test)
    accuracies.append(accuracy)


optimal_index = np.argmax(accuracies)
optimal_epsilon = epsilon_values[optimal_index]

print(f"Optimal Epsilon: {optimal_epsilon}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
14/14 ━━━━━━━━━━━━━

In [15]:
from sklearn.preprocessing import StandardScaler
noisy_X_train = add_laplace_noise(X_train, sensitivity, optimal_epsilon)
noisy_X_test = add_laplace_noise(X_test, sensitivity, optimal_epsilon)
sc=StandardScaler()
X_train_scaled = sc.fit_transform(noisy_X_train)
X_test_scaled = sc.transform(noisy_X_test)

In [16]:
model = SimpleMLP.build(X_train_scaled.shape[1])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(noisy_X_train, y_train, epochs=10, verbose=0)
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.74
